<a href="https://colab.research.google.com/github/paulinofm/tensorflow2/blob/master/Colab_3_Crear_una_Red_Neuronal_Artificial_en_TensorFlow_2_0_pfm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<p align="center">
  <img src="https://storage.googleapis.com/kaggle-datasets-images/2243/3791/9384af51de8baa77f6320901f53bd26b/dataset-cover.png" />
  Imagen cortesía de: https://www.kaggle.com/
</p>

## Paso 1: Instalar las dependencias y configurar el entorno de GPU

In [1]:
!pip install tensorflow-gpu==2.3.1

     |████████████████████████████████| 320.4MB 47kB/s 


## Paso 2: Importar las dependencias necesarias para el proyecto

In [3]:
import numpy as np
import datetime
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

In [4]:
tf.__version__

'2.3.1'

## Paso 3: Pre procesado de datos



### Cargar el dataset

In [6]:
#Cargar el dataset Fashion Mnist 
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


### Normalizar las imágenes

Se divide cada imagen en los conjunto de entrenamiento y  de testing entre el valor máximo de cada uno de los píxeles (255).

De este modo, cada píxel se hallará en el rango [0, 1]. Al normalizar las imágenes, nos aseguramos que nuestro modelo de RNA entrenará más rápidamente.

In [7]:
X_train = X_train / 255.0

In [8]:
X_test = X_test / 255.0

### Redimensionar el dataset

Como vamos a utilizar una red neuronal totalmente conectada, vamos a redimensionar los subconjuntos de entrenamiento y testing a formato de vector en lugar de en formato de matriz.

In [9]:
#Como cada imagen tiene 28x28 píxeles, usamos la función reshape en todo el dataset de entrenamiento para convertirlo 
# en vectores de tamaño [-1 (todos los elementos), anchura * altura]
X_train = X_train.reshape(-1, 28*28)

In [10]:
X_train.shape

(60000, 784)

In [11]:
#Redimensionamos el conjunto de testing del mismo modo
X_test = X_test.reshape(-1, 28*28)

## Paso 4: Construir la Red Neuronal Artificial

### Definir el modelo

Simplemente se define un objeto de modelo Sequential.

In [12]:
model = tf.keras.models.Sequential()

### Añadir la primera capa totalmente conectada (capa Densa)

Hyper-parametros de la capa:
- número de unidades/neuronas: 128
- función de activación: ReLU
- input_shape: (784, )

In [13]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784, )))

### Añadir una capa de Dropout 

Dropout es una técnica de Regularization donde aleatoriamente se asignan a ciertas neuronas de la red el valor cero. De este modo, mientras se entrena, estas neuronas no actualizarán sus valores. Al tener cierto porcentaje de neuronas sin actualizar, el proceso de entrenamiento toma más tiempo pero por contra tenemos menos posibilidades de sufrir overfitting.

In [14]:
model.add(tf.keras.layers.Dropout(0.15))

### Añado una segunda capa de 64 neuronas



In [15]:
model.add(tf.keras.layers.Dense(units=64, activation='relu', input_shape=(128, )))

### Añadir la capa de salida

- unidades: número de clases (10 en el caso del Fashion MNIST)
- función de activación: 'softmax' (probabilidades de cada clase)

In [16]:
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

### Compilar el modelo

- Optimizer: Adam
- Loss: Sparse softmax (categorical) crossentropy 

In [17]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
_________________________________________________________________


### Entrenar el modelo

In [21]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3085 - sparse_categorical_accuracy: 0.8850
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3001 - sparse_categorical_accuracy: 0.8888
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2905 - sparse_categorical_accuracy: 0.8918
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2835 - sparse_categorical_accuracy: 0.8924
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2765 - sparse_categorical_accuracy: 0.8952
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2687 - sparse_categorical_accuracy: 0.8978
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2641 - sparse_categorical_accuracy: 0.9010
Epoch 8/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2548 - sparse_categorical_accuracy: 0.9036
Epoch 9/10
1875/1875 [==========

### Evaluación del modelo y predicción

In [23]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3363 - sparse_categorical_accuracy: 0.8832


In [ ]:
print("Test accuracy: {}".format(test_accuracy))

## Paso 5 : Guardar el modelo

### Guardar la arquitectura (topoligía) de la red neuronal

In [24]:
model_json = model.to_json()
with open("fashion_model.json", "w") as json_file:
    json_file.write(model_json)

### Guardar los pesos de la red neuronal

In [25]:
model.save_weights("fashion_model.h5")